## Define dHCP pipeline

In [2]:
def dhcp_tractography_pipeline(subject, session, streamline_count=2000000, local_env=False):
    """
    For each `subject`, `session` pair run tractography pipeline.
    
    0. Perpare local environment
    1. Generate isotropic DWI files
    2. Realign DWI to anatomy using rigid transformation
    3. Estimate CSD response function
    4. Reassign ribbon values
    5. Generate five-tissue-type (5TT) segmented tissue image
    6. Tractography
    7. Create BIDS derivatives
    8. Upload to s3
    
    Parameters
    ----------
    subject : string
    session : string
    streamline_count : int
    local_env : boolean
    """
    
    import subprocess
    from os import makedirs
    from os.path import join
    from shutil import copyfile
    import json
    
    def print_quiet(string):
        """
        mrtrix doesn't provide a way to conviently supress progress messages without
        also supressing informational messages. 
        
        filter stderr/stdout before printing

        """
        import re
        
        # remove any line that looks like a progress message
        string = re.sub(r'.*\[[0-9 ]{3}\%\].*\n?', '', string)

        # remove empty lines
        string = re.sub(r'^\n$', '', string)
        
        # remove any leading or trailing whitespace
        string = string.strip()
        
        if string != "":
            print(string)
        
    
    ###############################################################################
    # Step 0: Prepare local environment
    ###############################################################################
    
    print('Step 0: Prepare local environment')
    
    makedirs('input', exist_ok=True)
    makedirs('output', exist_ok=True)
    
    def download_dhcp_files(subject, session):
        """
        Download anatomy and DWI files for tractography
        
        Parameters
        ----------
        subject : string
        session : string
        """
        import s3fs
        from os.path import exists, join
    
        fs = s3fs.S3FileSystem()
    
        anat_files = [
            f'sub-{subject}_ses-{session}_desc-drawem9_space-T2w_dseg.nii.gz',
            f'sub-{subject}_ses-{session}_desc-restore_T2w.nii.gz',
            f'sub-{subject}_ses-{session}_desc-ribbon_space-T2w_dseg.nii.gz'
        ]
        
        for anat_file in anat_files:
            if not exists(join('input', anat_file)):
                print('downloading:', anat_file)
                fs.get(
                    (
                        f'dhcp-afq/dhcp_anat_pipeline/sub-{subject}/ses-{session}/anat/'
                        f'{anat_file}'
                    ),
                    join('input', anat_file)
                )

        dwi_files = [
            f'sub-{subject}_ses-{session}_desc-preproc_dwi.bval',
            f'sub-{subject}_ses-{session}_desc-preproc_dwi.bvec',
            f'sub-{subject}_ses-{session}_desc-preproc_dwi.nii.gz',
            f'sub-{subject}_ses-{session}_desc-preproc_space-dwi_brainmask.nii.gz'
        ]

        for dwi_file in dwi_files:
            if not exists(join('input', dwi_file)):
                print('downloading:', dwi_file)
                fs.get(
                    (
                        f's3://dhcp-afq/dhcp_dmri_pipeline/sub-{subject}/ses-{session}/dwi/'
                        f'{dwi_file}'
                    ),
                    join('input', dwi_file)
                )
    
    download_dhcp_files(subject, session)
    
    ###############################################################################
    # Step 1: Generate isotropic DWI files
    ###############################################################################
    
    print('Step 1: Generate isotropic DWI files')
    
    def reslice_dwi(img_filename, new_zooms=(1.5, 1.5, 1.5)):
        """
        Creates an isotropic dwi file
        
        In dHCP dataset dwi images have voxel size [1.172 1.172 1.5]
        therefore by default scaling to 1.5
        
        Parameters
        ----------
        subject : string
        session : string
        """
        import os.path as op
        import nibabel as nib
        from dipy.align.reslice import reslice

        img = nib.load(img_filename)
        img_data = img.get_fdata()
        img_affine = img.affine
        img_zooms = img.header.get_zooms()[:3]
        new_data, new_affine = reslice(img_data, img_affine, img_zooms, new_zooms)
        new_img = nib.Nifti1Image(new_data, new_affine)

        # use original image name as basis for the resliced image name
        img_basename = op.basename(img_filename)
        desc_index = img_basename.rindex('_')
        img_name_start = img_basename[:desc_index]
        img_name_end = img_basename[desc_index:]
        new_img_filename = op.join('output', f'{img_name_start}_resliced{img_name_end}')
        
        print('saving resliced dwi:', new_img_filename)
        nib.save(new_img, new_img_filename)
    
    
    # dwi
    reslice_dwi(join('input', f'sub-{subject}_ses-{session}_desc-preproc_dwi.nii.gz'))

    # brainmask
    reslice_dwi(join('input', f'sub-{subject}_ses-{session}_desc-preproc_space-dwi_brainmask.nii.gz'))

    ###############################################################################
    # Step 2: Realign DWI to anatomy using rigid transformation
    ###############################################################################
    
    print('Step 2: Realign DWI to anatomy using rigid transformation')

    # convert NIFTI to MIF for MRtrix and generate b file
    # doing this because MRtrix `works better` with these formats
    mrconvert = subprocess.run(
        [
            'mrconvert',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.nii.gz',
            '-fslgrad',
            f'input/sub-{subject}_ses-{session}_desc-preproc_dwi.bvec',
            f'input/sub-{subject}_ses-{session}_desc-preproc_dwi.bval',
            '-export_grad_mrtrix',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrconvert.stdout)
    print_quiet(mrconvert.stderr)
    
    # generate b0 from dwi
    dwiextract = subprocess.run(
        [
            'dwiextract',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif',
            '-grad',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            '-'
            '-bzero'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    
    # don't bother printing out since piping to mrmath
    # print(dwiextract.stdout)
    # print(dwiextract.stderr)
    
    # pipe dwiextract to mrmath
    mrmath = subprocess.run(
        [
            'mrmath',
            '-',
            'mean',
            f'output/sub-{subject}_ses-{session}_desc-resliced_b0.mif',
            '-axis',
            '3'
        ],
        input = dwiextract.stdout,
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrmath.stdout)
    print_quiet(mrmath.stderr)
        
    # registered dwi to anat and extract transform
    mrregister = subprocess.run(
        [
            'mrregister',
            '-type',
            'rigid',
            f'output/sub-{subject}_ses-{session}_desc-resliced_b0.mif',
            f'input/sub-{subject}_ses-{session}_desc-restore_T2w.nii.gz',
            '-transformed',
            f'output/sub-{subject}_ses-{session}_desc-resliced_aligned_b0.mif'
            '-rigid',
            f'output/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrregister.stdout)
    print_quiet(mrregister.stderr)
    
    # apply transform to dwi
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.mif',
            '-grad',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            '-export_grad_mrtrix',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.b',
            '-linear',
            f'output/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    # export a NIFTI version (for pyafq) with FSL
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.mif',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.nii.gz',
            '-grad',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_dwi.b',
            '-export_grad_fsl',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bvec',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bval',
            '-linear',
            f'output/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    # apply transform to brainmask
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}_ses-{session}_desc-preproc_space-dwi_resliced_brainmask.nii.gz',
            f'output/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif',
            '-linear',
            f'output/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    # export a NIFTI version (for pyafq)
    mrtransform = subprocess.run(
        [
            'mrtransform',
            f'output/sub-{subject}_ses-{session}_desc-preproc_space-dwi_resliced_brainmask.nii.gz',
            f'output/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.nii.gz',
            '-linear',
            f'output/sub-{subject}_ses-{session}_desc-rigid_transform.txt'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mrtransform.stdout)
    print_quiet(mrtransform.stderr)
    
    ###############################################################################
    # Step 3: Estimate CSD response function
    ###############################################################################
    
    print('Step 3: Estimate CSD response function')
    
    # tissue response function
    dwi2response = subprocess.run(
        [
            'dwi2response',
            'dhollander'
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.mif',
            f'output/sub-{subject}_ses-{session}_desc-wm.txt',
            f'output/sub-{subject}_ses-{session}_desc-gm.txt',
            f'output/sub-{subject}_ses-{session}_desc-csf.txt',
            '-mask',
            f'output/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif',
            '-grad',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.b',
            '-voxels',
            f'output/sub-{subject}_ses-{session}_desc-voxels.mif',
            '-fa',
            '0.1'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(dwi2response.stdout)
    print_quiet(dwi2response.stderr)
    
    # CSD estimates
    dwi2fod = subprocess.run(
        [
            'dwi2fod',
            'msmt_csd',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.mif',
            f'output/sub-{subject}_ses-{session}_desc-wm.txt',
            f'output/sub-{subject}_ses-{session}_desc-wm.mif',
            f'output/sub-{subject}_ses-{session}_desc-csf.txt',
            f'output/sub-{subject}_ses-{session}_desc-csf.mif',
            '-mask',
            f'output/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif',
            '-grad',
            f'output/sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.b'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(dwi2fod.stdout)
    print_quiet(dwi2fod.stderr)
    
    # normalize
    mtnormalise = subprocess.run(
        [
            'mtnormalise',
            f'output/sub-{subject}_ses-{session}_desc-wm.mif',
            f'output/sub-{subject}_ses-{session}_desc-wm_norm.mif',
            f'output/sub-{subject}_ses-{session}_desc-csf.mif',
            f'output/sub-{subject}_ses-{session}_desc-csf_norm.mif',
            '-mask',
            f'output/sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.mif'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(mtnormalise.stdout)
    print_quiet(mtnormalise.stderr)

    ###############################################################################
    # Step 4: Reassign ribbon values
    ###############################################################################
    
    print('Step 4: Reassign ribbon values')

    def update_ribbon(subject, session, ventricle=43):
        """
        Ribbon file uses FreeSurfer definitions
        https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT

        NOTE: hemisphere designation does not matter as mrtrix merges
        using right hemisphere labels

        Parameters
        ----------
        subject : string
        session : string
        ventricle : int
        """

        from os.path import join
        import nibabel as nib
        import numpy as np

        drawem9 = nib.load(join('input', f'sub-{subject}_ses-{session}_desc-drawem9_space-T2w_dseg.nii.gz'))
        ribbon = nib.load(join('input', f'sub-{subject}_ses-{session}_desc-ribbon_space-T2w_dseg.nii.gz'))
        ribbon_data = ribbon.get_fdata()

        if ventricle == 43:
            # ventricles=5 - label as lateral ventricle left=4 or right=43
            augmentation_name = 'agmntd'
            ribbon_data[np.where(drawem9.get_fdata() == 5)] = 43
        elif ventricle == 0:
            augmentation_name = 'zero_vent_agmntd'
            # ventricles=5 - label as non-cerebrum=0
            ribbon_data[np.where(drawem9.get_fdata() == 5)] = 0
        else:
            # do not relabel ventricles
            augmentation_name = 'no_vent_agmntd'

        # brainstem=8 - label as white matter left=2 or right=41
        ribbon_data[np.where(drawem9.get_fdata() == 8)] = 41

        augmented_ribbon = nib.Nifti1Image(ribbon_data, ribbon.affine)
        ribbon_file = join('output', f'sub-{subject}_ses-{session}_desc-ribbon_space_{augmentation_name}-T2w_dseg.nii.gz')
        print('saving updated ribbon:', ribbon_file)
        nib.save(augmented_ribbon, ribbon_file)
        return ribbon_file

    ribbon_file = update_ribbon(subject, session)

    ###############################################################################
    # Step 5. Generate five-tissue-type (5TT) segmented tissue image
    ###############################################################################
    
    print('Step 5. Generate five-tissue-type (5TT) segmented tissue image')

    # 5tt
    fivettgen = subprocess.run(
        [
            '5ttgen',
            'freesurfer',
            '-lut',
            'FreeSurferColorLUT.txt' if local_env else '$FREESURFER_HOME/FreeSurferColorLUT.txt',
            ribbon_file,
            f'output/sub-{subject}_ses-{session}_desc-tt5.mif',
            '-nocrop'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(fivettgen.stdout)
    print_quiet(fivettgen.stderr)
    
    # gmwmi
    fivett2gmwmi = subprocess.run(
        [
            '5tt2gmwmi'
            f'output/sub-{subject}_ses-{session}_desc-tt5.mif',
            f'output/sub-{subject}_ses-{session}_desc-tt5_gmwmi.mif'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(fivett2gmwmi.stdout)
    print_quiet(fivett2gmwmi.stderr)
    
    ###############################################################################
    # Step 6. Tractography
    ###############################################################################
    
    print('Step 6. Tractography')

    tckgen = subprocess.run(
        [
            'tckgen',
            f'output/sub-{subject}_ses-{session}_desc-wm_norm.mif',
            '-algo',
            'IFOD1',
            '-backtrack',
            '-crop_at_gmwmi'
            '-seed_gmwmi',
            f'output/sub-{subject}_ses-{session}_desc-tt5_gmwmi.mif',
            '-act',
            f'output/sub-{subject}_ses-{session}_desc-tt5.mif',
            '-angle',
            '15',
            '-select',
            str(streamline_count),
            f'output/sub-{subject}_ses-{session}_desc-csd_tractography.tck',
            '-cutoff',
            '0.05'
        ],
        check = True,
        capture_output = True,
        text=True
    )
    print_quiet(tckgen.stdout)
    print_quiet(tckgen.stderr)
    
    ###############################################################################
    # Step 7. Create BIDS derivatives
    ###############################################################################
    
    print('Step 7. Create BIDS derivatives')
    
    makedirs(join('mrtrix', f'sub-{subject}', f'ses-{session}'), exist_ok=True)
    
    # NOTE: s3 already has this file, only necessary if intending to run locally
    if local_env and not exists(join('mrtrix', 'dataset_description.json')):
        dataset_description = {
            "Name" : "dHCP neonatal MRtrix derivatives",
            "PipelineDescription" : {
                "Name" : "dHCP neonatal MRtrix pipeline"
            },
            "BIDSVersion" : "1.4.0"
        }

        with open(join('mrtrix', 'dataset_description.json'), 'w') as f:
            json.dump(dataset_description, f)
        
    derivatives_files = [
        f'sub-{subject}_ses-{session}_desc-csd_tractography.tck',
        f'sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.nii.gz',
        f'sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.nii.gz',
        f'sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bvec',
        f'sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bval'
    ]
    
    for derivatives_file in derivatives_files:
        copyfile(
            join('output', derivatives_file),
            join('mrtrix', f'sub-{subject}', f'ses-{session}', derivatives_file)
        )
    
    ###############################################################################
    # Step 8. Upload to s3
    ###############################################################################

    print('Step 8. Upload to s3')

    # TODO upload input folder to s3
    
    # TODO upload output folder to s3
    
    # TODO upload mrtrix derivatives folder to s3

## Local

In [3]:
# first few subjects from dHCP dataset
subject_session_tuples = [
    ('CC00060XX03', '12501'), 
    ('CC00062XX05', '13801'),
    ('CC00063AN06', '15102'),
    ('CC00065XX08', '18600'),
    ('CC00066XX09', '19200')
]

In [ ]:
for subject_session_tuple in subject_session_tuples:
    print('Begin pipeline:', subject_session_tuple)
    dhcp_tractography_pipeline(subject_session_tuple[0], subject_session_tuple[1], local_env=True)

Begin pipeline: ('CC00060XX03', '12501')
Step 0: Prepare local environment
downloading: sub-CC00060XX03_ses-12501_desc-drawem9_space-T2w_dseg.nii.gz
downloading: sub-CC00060XX03_ses-12501_desc-restore_T2w.nii.gz
downloading: sub-CC00060XX03_ses-12501_desc-ribbon_space-T2w_dseg.nii.gz
downloading: sub-CC00060XX03_ses-12501_desc-preproc_dwi.bval
downloading: sub-CC00060XX03_ses-12501_desc-preproc_dwi.bvec
downloading: sub-CC00060XX03_ses-12501_desc-preproc_dwi.nii.gz
downloading: sub-CC00060XX03_ses-12501_desc-preproc_space-dwi_brainmask.nii.gz
Step 1: Generate isotropic DWI files
saving resliced dwi: output/sub-CC00060XX03_ses-12501_desc-preproc_resliced_dwi.nii.gz
saving resliced dwi: output/sub-CC00060XX03_ses-12501_desc-preproc_space-dwi_resliced_brainmask.nii.gz
Step 2: Realign DWI to anatomy using rigid transformation
mrconvert: [INFO] opening image "output/sub-CC00060XX03_ses-12501_desc-preproc_resliced_dwi.nii.gz"...
mrconvert: [INFO] Axes and transform of image "output/sub-CC000

dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /Users/bloomdt/NRDG/babyafq/dwi2response-tmp-TE9I8G/
dwi2response: Importing DWI data (/Users/bloomdt/NRDG/babyafq/output/sub-CC00060XX03_ses-12501_desc-preproc_resliced_aligned_dwi.mif)...
Command:  mrconvert /Users/bloomdt/NRDG/babyafq/output/sub-CC00060XX03_ses-12501_desc-preproc_resliced_aligned_dwi.mif /Users/bloomdt/NRDG/babyafq/dwi2response-tmp-TE9I8G/dwi.mif -strides 0,0,0,1 -grad /Users/bloomdt/NRDG/babyafq/output/sub-CC00060XX03_ses-12501_desc-preproc_resliced_aligned_dwi.b
          [?7l
dwi2response: Importing mask (/Users/bloomdt/NRDG/babyafq/output/sub-CC00060XX03_ses-12501_desc-preproc_space_dwi_resliced_aligned_brainmask.mif)...
Command:  mrconvert /Users/bloomdt/NRDG/babyafq/output/sub-CC00060XX03_ses-12501_desc-preproc_

dwi2fod: [WARNING] existing output files will be overwritten
dwi2fod: [INFO] opening image "output/sub-CC00060XX03_ses-12501_desc-preproc_resliced_aligned_dwi.mif"...
dwi2fod: [INFO] image "output/sub-CC00060XX03_ses-12501_desc-preproc_resliced_aligned_dwi.mif" opened with dimensions 100x100x64x300, voxel spacing 1.5x1.5x1.5x1, datatype Float32LE
dwi2fod: [INFO] opening image "output/sub-CC00060XX03_ses-12501_desc-preproc_space_dwi_resliced_aligned_brainmask.mif"...
dwi2fod: [INFO] image "output/sub-CC00060XX03_ses-12501_desc-preproc_space_dwi_resliced_aligned_brainmask.mif" opened with dimensions 100x100x64, voxel spacing 1.5x1.5x1.5, datatype Float32LE
dwi2fod: [INFO] found 300x4 diffusion gradient table
dwi2fod: [INFO] Diffusion gradient encoding data clustered into 3 non-zero shells and 20 b=0 volumes
dwi2fod: [INFO] initialising multi-tissue CSD for 2 tissue types, with 46 parameters
dwi2fod: [INFO] Multi-shell, multi-tissue CSD initialised successfully
dwi2fod: [INFO] creating im

saving updated ribbon: output/sub-CC00060XX03_ses-12501_desc-ribbon_space_agmntd-T2w_dseg.nii.gz
Step 5. Generate five-tissue-type (5TT) segmented tissue image
5ttgen: 
5ttgen: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
5ttgen: 
5ttgen: Generated scratch directory: /Users/bloomdt/NRDG/babyafq/5ttgen-tmp-7KN99D/
Command:  mrconvert /Users/bloomdt/NRDG/babyafq/output/sub-CC00060XX03_ses-12501_desc-ribbon_space_agmntd-T2w_dseg.nii.gz /Users/bloomdt/NRDG/babyafq/5ttgen-tmp-7KN99D/input.mif
          [?7l
          [?7l
Function: shutil.copyfile('/Users/bloomdt/NRDG/babyafq/FreeSurferColorLUT.txt', '/Users/bloomdt/NRDG/babyafq/5ttgen-tmp-7KN99D/LUT.txt')
5ttgen: Changing to scratch directory (/Users/bloomdt/NRDG/babyafq/5ttgen-tmp-7KN99D/)
Command:  labelconvert input.mif LUT.txt /usr/local/mrtrix3/share/mrtrix3/_5ttgen/FreeSurfer2ACT.txt indices.mif
          labelcon

clean up

In [ ]:
import shutil
shutil.rmtree('input', ignore_errors = True)
shutil.rmtree('output', ignore_errors = True)
shutil.rmtree('mrtrix', ignore_errors = True)

## AWS

In [ ]:
# TODO determine from s3
# subject_session_tuples

In [ ]:
import cloudknot as ck

TODO: ensure docker base image?

In [ ]:
dockerImage = ck.DockerImage(
    name = 'dhcp-baby-afq',
    func = dhcp_tractography,
    base_image = 'dhcp',
    overwrite = True
)